# Involve Malaysia - Data Science/Business Intelligence Test Module Solution

### Author: Bazil Muzaffar Kotriwala
### Timestamp: 06-Mar-18 7:09PM

In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import string
import re
import calendar
from dateutil.relativedelta import relativedelta
from difflib import SequenceMatcher

## Task 1: Number Summation

In [2]:
def sum_array(num_array):
    '''
    This function finds an array output such that output[i] is equal to the sum of all the elements of nums except nums[i].
    :param: An input array for e.g [1,2,3,4]
    :precondition: The number array must exist
    :return: A new array equal to the sum of all the elements of nums except nums[i].
    :complexity: Best Case = Worst Case = O(n), where n is the size of the list
    '''
    
    total_sum = 0
    summation_array = []
    
    # Get total sum of input array
    for number in num_array:
        total_sum += number
    
    # Calculate new number at each index and append it into the new list
    for except_num in num_array:
        summation_array.append(total_sum - except_num)
    
    return summation_array

test_array = [1,2,3,4]
sum_array(test_array)

[9, 8, 7, 6]

## Task 2: Sales Data Exploration and Analysis

In [3]:
# Part a: Read Sales data-set
weekly_sales_data = pd.read_csv('Sales_Transactions_Dataset_Weekly (1).csv')
weekly_sales_data.head()

,Product_Code,W0,W1,W2,W3,W4,W5,W6,W7,W8,...,Normalized 42,Normalized 43,Normalized 44,Normalized 45,Normalized 46,Normalized 47,Normalized 48,Normalized 49,Normalized 50,Normalized 51
0,P1,11,12,10,8,13,12,14,21,6,...,0.06,0.22,0.28,0.39,0.50,0.00,0.22,0.17,0.11,0.39
1,P2,7,6,3,2,7,1,6,3,3,...,0.20,0.40,0.50,0.10,0.10,0.40,0.50,0.10,0.60,0.00
2,P3,7,11,8,9,10,8,7,13,12,...,0.27,1.00,0.18,0.18,0.36,0.45,1.00,0.45,0.45,0.36
3,P4,12,8,13,5,9,6,9,13,13,...,0.41,0.47,0.06,0.12,0.24,0.35,0.71,0.35,0.29,0.35
4,P5,8,5,13,11,6,7,9,14,9,...,0.27,0.53,0.27,0.60,0.20,0.20,0.13,0.53,0.33,0.40


### Part b: Best Performing Product

In [4]:
# The best performing product is the product with the highest number of sales over the 52 weeks

# Read Sales data-set
weekly_sales_data = pd.read_csv('Sales_Transactions_Dataset_Weekly (1).csv')

# Finding total sold for each product across 52 weeks
weekly_sales_data['total_sold'] = weekly_sales_data.iloc[:,1:53].sum(axis=1)

# Finding best product with the highest volume of sales
best_product = weekly_sales_data.Product_Code[weekly_sales_data['total_sold'].idxmax()]
print('The best performing product is:', best_product)

The best performing product is: P409


### Part c: Most Promising Product (Emerging Product)

In [5]:
# To find the most promising product, we find avg sale in first 26 weeks and remaining 26 weeks for each product respectively
# The one with the highest rise in average sales will be the emerging product

# Read data-set
weekly_sales_data = pd.read_csv('Sales_Transactions_Dataset_Weekly (1).csv')

# Create 3 new columns
weekly_sales_data['first_26_weeks_avg'] = weekly_sales_data.iloc[:, 1:27].mean(axis=1)
weekly_sales_data['remaining_26_weeks_avg'] = weekly_sales_data.iloc[:, 27:53].mean(axis=1)
weekly_sales_data['change_in_avg'] = weekly_sales_data['remaining_26_weeks_avg'] - weekly_sales_data['first_26_weeks_avg']

# Find product with the max increase in average sales
product_max_rise = weekly_sales_data.Product_Code[weekly_sales_data['change_in_avg'].idxmax()]
max_rise_value = weekly_sales_data['change_in_avg'].max()
print('The most promising product is ' + product_max_rise + ' with an average rise in sales by ' + str(round(max_rise_value, 3)) +'%')

The most promising product is P674 with an average rise in sales by 3.038%


### Part d: Worst Performing Product (biweekly basis)

In [6]:
# To calculate worst performing product, we will first find total product amount sold on a biweekly basis instead of a weekly basis
# Then find the product with the average across all the biweeks for all products
# One with the lowest average will be the worst performing product

# Read data-set
weekly_sales_data = pd.read_csv('Sales_Transactions_Dataset_Weekly (1).csv')
weekly_sales_pcode = weekly_sales_data['Product_Code']

# Slice df to contain only week 0 - 51
weekly_sales_data = weekly_sales_data.iloc[:, 1:53]

# Create biweekly sales columns for each product
weekly_sales_data = pd.DataFrame(np.add.reduceat(weekly_sales_data.values, np.arange(len(weekly_sales_data.columns))[::2], axis=1))
weekly_sales_data['Product_Code'] = weekly_sales_pcode
weekly_sales_data.set_index('Product_Code', inplace=True)

# Finding the average of biweekly sales of each product
mean_products = weekly_sales_data.iloc[:].mean(axis=1)
mean_products = pd.DataFrame(mean_products)

# Finding worst product
worst_product = mean_products[0].idxmin()
print('The worst performing product based on lowest average biweekly sales is:', worst_product)

The worst performing product based on lowest average biweekly sales is: P215


### Part e: Outliers in the data

In [7]:
# Read data-set
weekly_sales_data = pd.read_csv('Sales_Transactions_Dataset_Weekly (1).csv')

# Create new dataframe containing mean, median, sd and sum of each week
weekly_descriptive_df = pd.DataFrame()
weekly_descriptive_df['weekly_mean'] = weekly_sales_data.iloc[:, 1:53].mean(axis=0)
weekly_descriptive_df['weekly_median'] = weekly_sales_data.iloc[:, 1:53].median(axis=0)
weekly_descriptive_df['weekly_std'] = weekly_sales_data.iloc[:, 1:53].std(axis=0)
weekly_descriptive_df['weekly_sum'] = weekly_sales_data.iloc[:, 1:53].sum(axis=0)

# Identify the weeks 
outlier_weeks = weekly_descriptive_df[weekly_descriptive_df.weekly_mean > np.percentile(weekly_descriptive_df.weekly_mean,95)]
print(outlier_weeks)
outlier_weeks = outlier_weeks.index.tolist()
print()
print(*['The following weeks are outlier weeks:'] + outlier_weeks)

     weekly_mean  weekly_median  weekly_std  weekly_sum
W15    10.045623            4.0   13.743655        8147
W16    10.033292            4.0   13.890316        8137
W24    10.166461            5.0   12.298777        8245

The following weeks are outlier weeks: W15 W16 W24


# Task 3: Job Posts Data Exploration and Analysis

In [8]:
# Part a: Read Job Post data-set
job_posts_df = pd.read_csv('data job posts.csv')
job_posts_df.head()

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\r\nJOB TI...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"To apply for this position, please submit a\r\...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,...,NaN,Please submit a cover letter and resume to:\r\...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\r\nJ...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\r\nPOSITION,...,NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\r\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\r\...,NaN,2004,1,False
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Please send cover letter and resume to Amy\r\n...,NaN,23 January 2004\r\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\r\nJOB TITLE: Software...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Successful candidates should submit\r\n- CV; \...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True


### Part b: Extract fields from jobpost column 

In [13]:
# Read data-set
job_posts_df = pd.read_csv('data job posts.csv')

# Create lists for each respective job post and remove \r\n
def process_jobposts():
    return job_posts_df['jobpost'].apply(lambda x: x.split('\r\n'))

# Store the respective positions in a dictionary
def create_field_df():
    field_dict = {'JOB TITLE': [], 'POSITION LOCATION': [], 'POSITION DURATION': [], 'JOB DESCRIPTION': [],
                  'JOB RESPONSIBILITIES': [],'REQUIRED QUALIFICATIONS': [], 'REMUNERATION': [], 'APPLICATION DEADLINE': [], 
                  'ABOUT COMPANY': []}
    
    for i in range(len(job_post_lists)):
        for j in range(len(job_post_lists[i])):
            if len(job_post_lists[i]) > 1 and job_post_lists[i][j] != '' and 'TITLE' in job_post_lists[i][j]:
                field_dict['JOB TITLE'].append(job_post_lists[i][j].replace('JOB TITLE:', '').replace('TITLE:', '').strip(' '))
            if len(job_post_lists[i]) > 1 and job_post_lists[i][j] != '' and 'LOCATION' in job_post_lists[i][j]:
                field_dict['POSITION LOCATION'].append(job_post_lists[i][j].replace('POSITION LOCATION:', '').replace('LOCATION:', '').strip(' '))
            if len(job_post_lists[i]) > 1 and job_post_lists[i][j] != '' and 'DURATION' in job_post_lists[i][j]:
                field_dict['POSITION DURATION'].append(job_post_lists[i][j].replace('POSITION DURATION:', '').replace('DURATION:', '').strip(' '))
            if len(job_post_lists[i]) > 1 and job_post_lists[i][j] != '' and 'DEADLINE' in job_post_lists[i][j]:
                field_dict['APPLICATION DEADLINE'].append(job_post_lists[i][j].replace('APPLICATION DEADLINE:', '').replace('DEADLINE:', '').strip(' '))
            if len(job_post_lists[i]) > 1 and job_post_lists[i][j] != '' and 'REMUNERATION' in job_post_lists[i][j]:
                field_dict['REMUNERATION'].append(job_post_lists[i][j].replace('REMUNERATION:', '').replace('REMUNERATION/ SALARY', '').strip(' '))  
    return field_dict  

job_post_lists = process_jobposts()
field_dict = create_field_df()

### Part c: Company with most job ads in past 2 years

In [10]:
# Fill missing values
job_posts_df['Company'] = job_posts_df['Company'].fillna('NULL')

# Convert date strings to date-time format
job_posts_df['date'] = pd.to_datetime(job_posts_df['date'], errors = 'coerce')

# Create data-frame containing all the data from the past 2 years
end_date = job_posts_df['date'][len(job_posts_df['date']) - 1]
start_date = yesterday = end_date - relativedelta(years=2)
jobs_2y_df = job_posts_df.loc[(job_posts_df['date'] > start_date) & (job_posts_df['date'] <= end_date)]

# Clean last two years data
def remove_punctuation():
    return jobs_2y_df['Company'].apply(lambda x: re.sub('[^A-Za-z0-9]+', '', x.lower()))

def remove_tags():
    return comp_df['Company'].apply(lambda x: x.replace('llc', '').replace('ltd', '')
                                    .replace('consulting', '').replace('co', '').replace('cjsc', '').replace('TITLE:', '')
                                    .replace('csjc', ''))

# Call functions
company_freq = remove_punctuation()
comp_df = pd.DataFrame(company_freq)
tags = remove_tags()
comp_new_df = pd.DataFrame(remove_tags())

# Find company with the max number of job posts
max_company = comp_new_df.groupby(['Company']).size().idxmax()
print('The company with the most job ads in the past 2 years is:', max_company)

The company with the most job ads in the past 2 years is: mentorgraphicsdevelopmentservices


### Part d: Month with largest number of job ads

In [11]:
# Convert date strings to date-time format
job_posts_df['date'] = pd.to_datetime(job_posts_df['date'], errors = 'coerce')

# Index rows by date
job_posts_df.index = pd.to_datetime(job_posts_df.date)

# Create column depicting month number for each date
job_posts_df['month'] = pd.DatetimeIndex(job_posts_df['date']).month

# Group by month number with respective job ads frequency
month_number = int(job_posts_df.groupby(['month']).size().idxmax())

# Extract month name from month number
month_name = calendar.month_name[month_number]
print('The month with the largest number of job ads is:', month_name)

The month with the largest number of job ads is: September


# Task 4: String Similarity

In [12]:
# Read test data-set
text_test_df = pd.read_csv('test.csv')

# Find similarity score between description_x and description_y
text_test_df['similarity_score'] = text_test_df.apply(lambda x: SequenceMatcher(None, x[1].strip(), x[2].strip()).ratio(), axis=1)
text_test_df.head()

,test_id,description_x,description_y,same_security,similarity_score
0,0,semtech corp,semtech corporation,NaN,0.774194
1,1,vanguard mid cap index,vanguard midcap index - a,NaN,0.893617
2,2,spdr gold trust gold shares,spdr gold trust spdr gold shares,NaN,0.915254
3,3,vanguard total bond index adm,vanguard total bond market index,NaN,0.819672
4,4,oakmark international fund class i,oakmark international cl i,NaN,0.866667
